# Setup

In [ ]:
from google.colab import drive
import os

drive.mount("/content/drive")

project_dir = "/content/drive/MyDrive/PEFT-optimization/"
os.makedirs(project_dir, exist_ok=True)
os.chdir(project_dir)

output_dir = os.path.join(project_dir, "outputs") #Stores checkpoints during training. Good if something crashes
merged_dir = "/content/merged_model" #This is a large file that we don't need to save as we just use it for conversion to gguf
gguf_dir = "/content/gguf" #Stores the converted GGUF model locally as we will upload to HF immediately after
lora_dir = os.path.join(project_dir, "lora_model") #Stores the trained LoRA weights. Can use this to recreate merged model if needed

# Persistent directories
os.makedirs(lora_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

# Ephemeral directories
os.makedirs(merged_dir, exist_ok=True)
os.makedirs(gguf_dir, exist_ok=True)


Install unsloth on colab:

In [ ]:
%%capture
!pip install python-dotenv
!pip install -U "transformers" "huggingface_hub" # Install latest transformers and huggingface_hub to push GGUF. As recommended by Adrian
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git@nightly git+https://github.com/unslothai/unsloth-zoo.git

Get .env contents and keys

In [ ]:
from dotenv import load_dotenv
from google.colab import userdata

def load_hf_config():
    """
    Load Hugging Face credentials from:
      1. Colab userdata secrets (if running on Colab)
      2. Local .env file (if running locally)
      3. Existing environment variables (fallback)
    """
    # --- Check if running in Colab ---
    try:
        
        # If this works, we are in Colab.
        username = userdata.get("HF_USERNAME")
        token = userdata.get("HF_TOKEN")
        repo_name = userdata.get("HF_REPO_NAME") or "PEFT-optimization"

        if username:
            os.environ["HF_USERNAME"] = username
        if token:
            os.environ["HF_TOKEN"] = token
        if repo_name:
            os.environ["HF_REPO_NAME"] = repo_name

        print("Loaded HuggingFace credentials from Colab secrets.")
    
    except Exception:
        # Not in Colab → load from .env
        load_dotenv()  # loads .env into os.environ if present
        print("Loaded HuggingFace credentials from .env")

    # Final read
    username = os.getenv("HF_USERNAME")
    repo_name = os.getenv("HF_REPO_NAME", "PEFT-optimization")
    token = os.getenv("HF_TOKEN")

    # Validate
    missing = []
    if not username:
        missing.append("HF_USERNAME")
    if not token:
        missing.append("HF_TOKEN")

    if missing:
        raise ValueError(f"Missing required environment variables: {missing}")

    return username, repo_name, token

print("Testing credentials early...")
username, repo_name, token = load_hf_config()
print("HF credentials OK, safe to continue!")


Download the model. The fourbit models download faster via unsloth, but we can change the model in `model_name`. Kept the rest of settings default

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit" # NEW! Llama 3.3 70B!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit", # Changed to a model from fourbit_models list. Smaller one also
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

Set up PEFT (LoRA). Settings default for now. Can tune later

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

# Data Prep

This section cleans up the training prompts from the FineTome dataset so that they match what llama expects.

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

from datasets import load_dataset
dataset = load_dataset("mlabonne/FineTome-100k", split = f"train[:10000]") #Reduced training size to 10k to save memory

In [ ]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

# Train Model

Create a trainer for the model. Using Hugginfaces default trainer

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = True, # To make training faster, set this to True
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        #num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 600, #Changing to 600 for more in depth training (v2) (v1 used 50 steps)
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit", # Use "adamw_torch" when running locally, "adamw_8bit" works well on Colab
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = output_dir,
        report_to = "none", # Use this for WandB etc
        #Below is where we save the weights as we go
        save_strategy = "steps",
        save_steps = 100,
        save_total_limit = 2, #Saving only the last 2 checkpoints
    ),
)

Below we format the data so that we only train on the expected responses. The model doesn't benefit of training on what the user inputs, we just want to refine how it predicts the next word in the outputs, so we remove the user inputs from the training set. 

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

## Show GPU Memory Stats

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

Actually running the trainer. Resuming from checkpoint if it exists

In [ ]:
from transformers.trainer_utils import get_last_checkpoint

checkpoint = None
if os.path.isdir(output_dir):
    try:
        checkpoint = get_last_checkpoint(output_dir)
        if checkpoint is not None:
            print(f"Resuming training from checkpoint: {checkpoint}")
        else:
            print(f"No checkpoint found in '{output_dir}'. Starting from scratch.")

    except Exception as e:
        print(f"Could not get checkpoint from '{output_dir}': {e}")
        checkpoint = None

trainer_stats = trainer.train(resume_from_checkpoint=checkpoint)


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

# Saving Tuned Model

## Saving just LoRA Adapter

In [ ]:
model.save_pretrained(lora_dir) # Local saving
tokenizer.save_pretrained(lora_dir)

#Save to HuggingFace Hub
username, repo_name, token = load_hf_config()
hf_repo_id = f"{username}/{repo_name}"

print(f"Pushing adapter to: {hf_repo_id}")

model.push_to_hub(hf_repo_id, token=token)
tokenizer.push_to_hub(hf_repo_id, token=token)


## Loading LoRA Adapter that we just saved

In [ ]:
# Disabled for now. This was an example included in the template
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

    messages = [
        {"role": "user", "content": "Describe a tall tower in the capital of France."},
    ]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    ).to("cuda")

    from transformers import TextStreamer
    text_streamer = TextStreamer(tokenizer, skip_prompt = True)
    _ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                    use_cache = True, temperature = 1.5, min_p = 0.1)

### GGUF / llama.cpp Conversion

Using llama.cpp for GGUF conversion instead of Unsloth as that ran out of memory on Colab

In [ ]:
# Merge LoRA into a 16-bit HF model for llama.cpp conversion
print("Merging LoRA into 16-bit base model...")

model.save_pretrained_merged(
    merged_dir,
    tokenizer,
    save_method="merged_16bit",
    maximum_memory_usage=0.5,  # fraction of GPU memory; tweak if needed
)

print(f"Merged model saved to: {merged_dir}")


In [ ]:


# Clone llama.cpp if not already there
if not os.path.isdir("llama.cpp"):
    !git clone https://github.com/ggerganov/llama.cpp.git

# Choose quantization
quantization_method = "q4_k_m"  # good default for CPU inference
gguf_filename = os.path.join(
    gguf_dir,
    f"llama-3.2-1b-finetuned-{quantization_method}_v2.gguf"
)


# Convert HF model in 'merged_model' to GGUF
!python llama.cpp/convert_hf_to_gguf.py {merged_dir} \
    --outfile {gguf_filename} \
    --outtype {quantization_method}

print(f"GGUF file written to: {gguf_filename}")


In [ ]:
from huggingface_hub import HfApi

username, repo_name, token = load_hf_config()
gguf_repo_id = f"{username}/{repo_name}"

api = HfApi()
api.create_repo(repo_id=gguf_repo_id, exist_ok=True, token=token)

api.upload_file(
    path_or_fileobj=gguf_filename,
    path_in_repo=os.path.basename(gguf_filename),
    repo_id=gguf_repo_id,
    token=token,
)

print(f"Uploaded GGUF model to https://huggingface.co/{gguf_repo_id}")


In [ ]:

if False: #Turned off for now
    # Save to q4_k_m GGUF
    model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")

    # Push to HuggingFace Hub GGUF
    username, repo_name, token = load_hf_config()
    model.push_to_hub_gguf(
        f"{username}/{repo_name}",
        tokenizer,
        quantization_method="q4_k_m",
        token=token,
    )


# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )